In [1]:
import pandas as pd
df=pd.read_excel('ForestProductKeywords.xlsx',header=None,encoding="utf-8")

In [2]:
df = df.dropna(subset=[1])
df = df.dropna(how='all')


In [3]:
# fill in the common names
df['id'] = None
prev = None
id = 1
prev_id = None
for i,row in  df.iterrows():
    cond = False
    if row[0] is None or type(row[0]) != str :
        cond = True
    elif type(row[0]) ==str and row[0] == '':
        cond = True
    if cond == False :
        res = row[0]
        res = res.replace('\n','')
        res = res.strip()
        df.loc[i,0] = res
        prev = res
        df.loc[i,'id'] = id
        prev_id = id
        id += 1
    else :
        df.loc[i,0] = prev
        df.loc[i,'id'] = prev_id

df.head(10)


,0,1,2,3,id
0,Abarco,Cariniana pyriformis Miers.,Venezuela,Bacu,1
1,Abura,Hallea ciliata Leroy\n(Syn. Mitragyna ciliata ...,Angola\nCameroon\nCongo\nCôte d’Ivoire\nEquato...,Mivuku\nElolom\nVuku\nBahia\nElelon\nElelom Nz...,2
2,Acacia,Acacia auriculiformis A.Cunn. ex Benth.\n \nAc...,Australia\n \nIndonesia\n \nMalaysia\nPapua Ne...,"Black Wattle,\nBrown Salwood\nMangge Hutan,\nT...",3
3,Acajou d’Afrique,Khaya spp.\n \nKhaya ivorensis A. Chev.\n(Syn....,Angola\nCameroon\nCôte d’Ivoire\nEquatorial Gu...,Undia Nunu\nN’Gollon\nAcajou Bassam\nCaoba del...,4
4,Acajou d’Afrique,Khaya anthotheca C. DC.,Angola\nCameroon\nCongo\nCôte d’Ivoire\n \nGha...,"N’Dola\nMangona\nN’Dola\nAcajou Blanc,\nAcajou...",4
5,Acajou d’Afrique,Khaya grandifoliola C. DC.,Côte d’Ivoire\n \nNigeria\n \nUganda\n \nFranc...,"Acajou à Grandes Feuilles\nAkuk,\nBenin Mahoga...",4
6,Adjouaba,Dacryodes klaineana (Pierre) H. J. Lam\n(Syn. ...,Dem. Rep. of the Congo\nCongo\nGabon\n,"Mouguengueri\nSafukala\nAssia,\nIgaganga,\nOss...",5
7,Afina,Strombosia glaucescens Engl.\n \nStrombosia pu...,Côte d’Ivoire\nNigeria,"Poe\nItako,\nOtingbo",6
8,Afrormosia,Pericopsis elata Van Meeuwen\n(Syn. Afrormosia...,Cameroon\nCentral African Republic\nCôte d’Ivo...,"Obang\n \nObang\nAssamela\nKokrodua\nOle,\nBoh...",7
11,Aielé,Canarium schweinfurtii Engl.\n,Angola\nCameroon\nCentral African Republic\nCo...,"M’bili\nAbel\n \nGberi\nM’bili\nAbeul,\nOvili\...",8


In [4]:
def parse_sp(text):
    res = []
    parts = text.split('Syn')
    for item in parts:
        item = item.replace(')','')
        item = item.replace('(','')
        item = item.strip('.')
        item = item.strip()
        parts2 =  item.split(' ')
        r = ' '.join(parts2[0:2])

        if r != '':
            res.append(r)
    res = ';'.join(res)
    return res


In [5]:
def process_sp_list(sp_sent):
    genus_list = []
    sp_list = []
    sp_sent = sp_sent.replace('\xa0','')
    sp_parts = sp_sent.split('\n')
    for _sp in sp_parts:
        if _sp == '' :
            continue
        if (' spp.' in _sp) or (' sp.' in _sp) or (' ' not in _sp and _sp[0].isupper()):
            if 'Syn' in _sp:
                r = parse_sp(_sp)
                r = r.split(' ')[0]
            else:
                r = _sp.split(' ')[0]
            genus_list.append(r)
        else :
            res = parse_sp(_sp)
            sp_list.append(res)
    
    genus_list = list(set(genus_list))
    sp_list = list(set(sp_list))
    return genus_list , sp_list

def get_common_names(text):
    if type(text) != str :
        return []
    text = text.replace('\xa0','')
    text = text.split('\n')
    res = []
    for text_part in text:
        parts = text_part.split(',')
        for t in parts :
            if t == '':
                continue
            if '(' in t and ')' in t :
                # extract parts in ( )
                tmp = t[t.index('(')+1:t.index(')')]
                res.append(tmp)
                t = t[0:t.index('(')]
            t = t.strip(',')
            t = t.strip(' ')
            res.append(t)
            
    return res

def get_pilot_names(text):
    text = text.replace('\xa0','')
    text = text.strip()
    res = []
    if '(' in text and ')' in text :
        tmp = text[text.index('(')+1:text.index(')')]
        tmp = tmp.split(',')
        tmp = [ _t.strip() for _t in tmp ]
        res.extend(tmp)
        text = text[0:text.index('(')]
    text = text.strip()
    res.append(text)
    return res

In [6]:
def get_loc_data(text):
    hardcoded_replacements = {
        'Central and South America': 'Central America,South America',
        'Trinidad and Tobago': 'Trinidad-Tobago',
        'West and Central Africa' : 'West Africa,Central Africa'
    }
    if type(text) != str :
        return None
    text = text.replace('\xa0','')
    
    for k,rep in hardcoded_replacements.items():
        text = text.replace(k,rep)
        
    text = text.split('\n')
    tmp = []
    for t in text:
        if t != '':
            t = t.replace(' and',',')
            t = t.split(',')
            t = [ _t.strip() for _t in t ]
            tmp.extend(t)
            
    res = tmp
    res = ';'.join(res)
    return res

In [7]:
new_df = pd.DataFrame(columns =['genus','species','common_name','range_state'])

for i,row in df.iterrows():
    sp_list = row[1]
    genus_list, sp_list = process_sp_list(sp_list)
    pilot_name = row[0]
    comm_names = row[3]
    loc_data = get_loc_data(row[2])
    if len(genus_list) > 0:
        for gn in genus_list:
            # add in genus with the common name as pilot name
            tmp_dict ={
                'genus':gn,
                'species':None,
                'common_name':pilot_name,
                'range_state':loc_data
            }
            new_df = new_df.append(tmp_dict, ignore_index=True)
            
    cn_list = get_common_names(row[3])
    pilot_name = get_pilot_names(pilot_name)
    cn_list.extend(pilot_name)
    cn_list = ';'.join(cn_list)
   
    for s in sp_list:
        gn = s.split(' ')[0]
        sp = s.split(' ')[1]
        tmp_dict ={
                'genus':gn,
                'species':sp,
                'common_name':cn_list,
                'range_state':loc_data
        }
#         print ('>> ', tmp_dict)
        new_df = new_df.append(tmp_dict, ignore_index=True)



In [8]:
#Process the country names to ISO3166
from iso3166 import countries
from pprint import pprint
replacement_df = pd.read_excel('Regions_Countries.xlsx')

region_dict = {}
for i,row in replacement_df.iterrows():
    region = row['Region']
    cnt = row['Countries']
    cnt = cnt.split(';')
    cnt = [c.strip() for c in cnt]
    cnt = ';'.join(cnt)
    region_dict[region] = cnt



In [9]:
def expand_locs(row):
    global region_dict
    locs = row['range_state']
    if type(locs) != str:
        return None
    
    locs = locs.split(';')
    res = []
    for l in locs :
        if l in region_dict.keys():
            l = region_dict[l]
            l = l.split(';')
            res.extend(l)
        else:
            res.append(l)
    res = ';'.join(res)
    return res       
        
new_df['range_state'] = new_df.apply(expand_locs, axis = 1)   

In [10]:
country_code_list = {
    'Surinam' : 'SR',
    'Congo' : 'CG',
    'Congo' : 'CG',
    'Burkina-Faso' : 'BF',
    'Venezuela' : 'VE',
    'Sri-Lanka' : 'LK',
    'French Guinea' : 'GF',
    'Bolivia':'BO',
    'Laos' : 'LA',
    'Tanzania' : 'TZ',
    "Côte d’Ivoire" : 'CI',
    'Vietnam' : 'VN',
    'Dominican Rep.' : 'DO',
    'Costa-Rica' : 'CR',
    'UK' : 'GB',
    'Trinidad-Tobago' : 'TT',
    'Ivory Coast': 'CI',
    'Saint Helena ,Ascension and Tristan da Cunha' : 'SH',
    'The Gambia' : 'GM',
    'Sint Maarten': 'SX',
    'Republic of the Congo':'CG',
    'Tuvalu': 'TV',
    'Nauru': 'NR',
    'Saint Martin' : 'MF',
    'United States Virgin Islands' : 'VI',
    'Cook Islands (NZ)' : 'CK',
    'Federated States of Micronesia' : 'FM',
    'Norfolk Island Niue' : 'NF',
    'Niue' : 'NU',
    'Madeira' : 'PT',
    'Ceuta' : 'ES',
    'North Korea' : 'KP',
    'Canary Islands' : 'ES',
    'East Timor': 'TL',
    'Syria':'SY',
    'South Korea' : 'KR',
    'São Tomé and Príncipe':'ST',
    'Macau': 'MO',
    'Macao': 'MO',
    'Iran': 'IR',
    'Beliza': 'BZ',
    'Melilla': 'ES',
    'Falkland Islands' : 'FK' ,
    'British Virgin Islands': 'VG' ,
    'Cape Verd' : 'CV' ,
    'Dem. Rep. Of the Congo' : 'CD' ,
    'Dem. Rep of the Congo' : 'CD',
    'Dem. Rep. of the Congo': 'CD',
    'Democratic Republic of the Congo' : 'CD',
    'Pitcairn Islands' : 'PN',
    'Caribbean Netherlands' : 'BQ',
    'Cape Verde' : 'CV'
}

countries_list = []
tmp = new_df['range_state']
for r in tmp :
    if r is None or r == '':
        continue
    r = r.split(';')
    countries_list.extend(r)

countries_list = list(set(countries_list))

for c in countries_list:
    try:
        res = countries.get(c)
        country_code_list[c] = res.alpha2
    except :
        pass
#         if c in country_code_list.keys():
#             print (country_code_list[c])
#         else :
#             print ( "[ Error ] , manually insert in correction_list >> " , c )





In [11]:
def replace_locs_with_iso(row):
    global country_code_list
    
    def get_iso(loc):
        try:
            res = countries.get(loc)
            country_code_list[loc] = res.alpha2
            return res.alpha2 
        except :
            if c in country_code_list.keys():
                return country_code_list[loc]
    locs = row['range_state']
   
    if type(locs) != str or locs == '' :
        return None
    
    locs = locs.split(';')
    iso_codes = [ get_iso(l) for l in locs]
    res = ';'.join(iso_codes)
    return res


new_df['range_state'] = new_df.apply(replace_locs_with_iso, axis = 1)     


In [12]:
new_df = new_df.sort_values(by=['genus','species'])
new_df.to_csv('ForeestKeywordsProcessed_v1.csv')

In [64]:
# Merge duplicates

df = pd.DataFrame(columns=['genus', 'species', 'common_name', 'range_state'])
tmp_df = pd.DataFrame(new_df,copy=True)

def rectify_None_vals(row):
    if row['species']is None or type(row['species'])!= str:
        return '-'
    else :
        return row['species']
tmp_df['species'] = tmp_df.apply(rectify_None_vals,axis=1)



In [65]:
skip = 0
for i,row in tmp_df.iterrows():
    sp = row['species']
    gn = row['genus']
    if skip > 0 :
        skip -=1 
        continue
    tmp = tmp_df.loc[(tmp_df['species'] == sp) & (tmp_df['genus'] == gn)]
    if len(tmp) > 1:
        skip = len(tmp)-1
        print ('----')
       # Join
        res_cn = []
        res_rs = []
        for j, _row in tmp.iterrows():
            print(_row)
            c = _row['common_name']
            r = _row['range_state']
            if type(c) == str :
                c = c.split(';')
                res_cn.extend(c)
            if type(r) == str :
                r = r.split(';')
                res_rs.extend(r)
        
        print(res_rs)
        print(res_cn)
        if len(res_rs) > 0 :
            res_rs = list(set(res_rs))
            res_rs = ';'.join(res_rs)
        else:
            res_rs = None
        
        if len(res_cn) > 0 :
            res_cn = list(set(res_cn))
            res_cn = ';'.join(res_cn)
        else:
            res_cn = None
            
        if sp == '-' :
            sp = None
            
        tmp_dict = {
            'genus' : gn,
            'species' : sp,
            'common_name' : res_cn,
            'range_state' : res_rs
        }
        
        df = df.append(tmp_dict,ignore_index=True)
    else :
        tmp = tmp.reset_index()
       
        tmp_dict = {
            'genus' : gn,
            'species' : sp,
            'common_name' : row['common_name'],
            'range_state' : row['range_state']
        }
        print (tmp_dict)
        df = df.append(tmp_dict,ignore_index=True)
        
df.head(10)

{'genus': 'Abarema', 'species': 'jupunba', 'common_name': 'Ingarana;Tento Azul;Huruasa', 'range_state': 'GY'}
{'genus': 'Acacia', 'species': 'auriculiformis', 'common_name': 'Black Wattle;Brown Salwood;Mangge Hutan;Tongke Hutan;Kayu Safoda;Arr;Kra Thin Tepa;Brown Salwood;Black Wattle;Brown Salwood;Black Wattle;Acacia', 'range_state': 'AU;ID;MY;PG;TH;GB;US'}
{'genus': 'Acacia', 'species': 'mangium', 'common_name': 'Black Wattle;Brown Salwood;Mangge Hutan;Tongke Hutan;Kayu Safoda;Arr;Kra Thin Tepa;Brown Salwood;Black Wattle;Brown Salwood;Black Wattle;Acacia', 'range_state': 'AU;ID;MY;PG;TH;GB;US'}
{'genus': 'Adina', 'species': 'cordifolia', 'common_name': 'Khvao;Kwao;Haldu;Lasi;Thom;Meraga;Hnaw;Adina;Haldu;Kolon;Kwao;Tong Lueang;Gao-Vang;Haldu', 'range_state': 'KH;IN;ID;LA;MY;MM;PH;LK;TH;VN'}
{'genus': 'Adina', 'species': 'microcephala', 'common_name': 'Matumi;Rhodesian Redwood;Mugonha', 'range_state': 'DZ;ES;ES;EG;LY;PT;ES;MA;SD;TN;EH;BI;KM;DJ;ER;ET;KE;MG;MW;MU;YT;MZ;RE;RW;SC;SO;SS;TZ;U

,genus,species,common_name,range_state
0,Abarema,jupunba,Ingarana;Tento Azul;Huruasa,GY
1,Acacia,auriculiformis,Black Wattle;Brown Salwood;Mangge Hutan;Tongke...,AU;ID;MY;PG;TH;GB;US
2,Acacia,mangium,Black Wattle;Brown Salwood;Mangge Hutan;Tongke...,AU;ID;MY;PG;TH;GB;US
3,Adina,cordifolia,Khvao;Kwao;Haldu;Lasi;Thom;Meraga;Hnaw;Adina;H...,KH;IN;ID;LA;MY;MM;PH;LK;TH;VN
4,Adina,microcephala,Matumi;Rhodesian Redwood;Mugonha,DZ;ES;ES;EG;LY;PT;ES;MA;SD;TN;EH;BI;KM;DJ;ER;E...
5,Afrormosia,elata,Obang;Obang;Assamela;Kokrodua;Ole;Bohala;Mohol...,CM;CF;CI;GH;CD;FR
6,Afzelia,africana,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
7,Afzelia,bella,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
8,Afzelia,bipindensis,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US
9,Afzelia,cuanzensis,N’kokongo Uvala;M’Banga;Doussié;N’Kokongo;Ling...,AO;CM;CG;CI;GH;MZ;NG;SN;SL;TZ;CD;DE;PT;GB;US


In [66]:
def remove_placeholder(row):
    if row['species'] == '-' :
        return None
    else :
        return row['species']

df['species'] = df.apply(remove_placeholder,axis=1)
df.to_csv('ForestKeywordsProcessed_v1.csv')       